In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from PIL import Image
from sklearn import metrics

In [2]:
df = pd.read_csv('data/train.csv').dropna()
df.drop(columns=['center_id', 'patient_id', 'image_num'], inplace=True)
df.head()

,image_id,label
0,006388_0,CE
1,008e5c_0,CE
2,00c058_0,LAA
3,01adc5_0,LAA
4,026c97_0,CE


In [3]:
CE_count = df.where(df['label'] == 'CE').count()[0]
CE_count

547

In [4]:
LAA_count = df.where(df['label'] == 'LAA').count()[0]
LAA_count

207

In [5]:
cutoff = CE_count if CE_count < LAA_count else LAA_count
cutoff

207

In [6]:
CE = df.where(df['label'] == 'CE').dropna().head(cutoff)
LAA = df.where(df['label'] == 'LAA').dropna().head(cutoff)

In [7]:
data = pd.concat([CE, LAA]).dropna()
data.head(cutoff*2)

,image_id,label
0,006388_0,CE
1,008e5c_0,CE
4,026c97_0,CE
6,029c68_0,CE
7,032f10_0,CE
...,...,...
740,fd684f_0,LAA
745,fde9ba_0,LAA
750,fe9bec_0,LAA
752,ffec5c_0,LAA


In [8]:
images = np.array([np.array(Image.open(f'jpgs/{img_id}.jpg')) for img_id in data['image_id']])
images.shape

(414, 512, 512)

In [9]:
enc_labels = np.zeros(shape=(414, 2))
for i, label in enumerate(data['label']):
    enc_labels[i][1 if label == 'CE' else 0] = 1

In [10]:
keras.utils.set_random_seed(1234)
ConvNN = keras.Sequential()
ConvNN.bias = 0.5
ConvNN.add(keras.layers.Input((512, 512)))
ConvNN.add(keras.layers.Rescaling(1.0/255))

#ConvNN.add(keras.layers.Reshape((512, 512)))

ConvNN.add(keras.layers.Convolution1D(256, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Flatten())
ConvNN.add(keras.layers.Dense(262144, activation='relu'))
ConvNN.add(keras.layers.Dense(512, activation='relu'))
ConvNN.add(keras.layers.Dense(256,  activation='relu'))

ConvNN.add(keras.layers.Dropout(.5))

ConvNN.add(keras.layers.Dense(2, activation='softmax'))
ConvNN.output_shape

(None, 2)

In [11]:
# Define the checkpoint path
checkpoint_path = "checkpoint/model_checkpoint.h5"

import os
# Define the ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    checkpoint_path,
    save_best_only=False,  # Save only the best model
    monitor='val_loss',  # Monitor validation loss for saving the best model
    mode='min',  # 'min' mode for loss (you can change it to 'max' for accuracy)
    verbose=1  # Print a message when a new best model is saved
)

# Load the model weights if a checkpoint file exists
if os.path.exists(checkpoint_path):
    ConvNN.load_weights(checkpoint_path)

In [12]:
ConvNN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [13]:
# Continue training from a specific epoch
epochs_completed = 0  # Change this to the number of epochs already completed
total_epochs = 25  # The total number of epochs (including completed ones)
# total_epochs = 25  # The total number of epochs (including completed ones)

ConvNN.fit(
    images, 
    enc_labels, 
    initial_epoch=epochs_completed,
    epochs=total_epochs, 
    callbacks = [model_checkpoint])

Epoch 1/25
13/13 [==============================] - ETA: 0s - loss: 3.3777 - accuracy: 0.5097
Epoch 1: saving model to checkpoint\model_checkpoint.h5
13/13 [==============================] - 66s 5s/step - loss: 3.3777 - accuracy: 0.5097
Epoch 2/25
13/13 [==============================] - ETA: 0s - loss: 0.6967 - accuracy: 0.5097
Epoch 2: saving model to checkpoint\model_checkpoint.h5
13/13 [==============================] - 82s 7s/step - loss: 0.6967 - accuracy: 0.5097
Epoch 3/25
13/13 [==============================] - ETA: 0s - loss: 0.6851 - accuracy: 0.5411
Epoch 3: saving model to checkpoint\model_checkpoint.h5
13/13 [==============================] - 71s 6s/step - loss: 0.6851 - accuracy: 0.5411
Epoch 4/25
13/13 [==============================] - ETA: 0s - loss: 0.6644 - accuracy: 0.6014
Epoch 4: saving model to checkpoint\model_checkpoint.h5
13/13 [==============================] - 3619s 301s/step - loss: 0.6644 - accuracy: 0.6014
Epoch 5/25
13/13 [==============================

In [14]:
keras.utils.set_random_seed(1234)
ConvNN = keras.Sequential()
ConvNN.bias = 0.5
ConvNN.add(keras.layers.Input((512, 512)))
ConvNN.add(keras.layers.Rescaling(1.0/255))

ConvNN.add(keras.layers.RandomFlip(
    mode="horizontal_and_vertical", seed=1234
))




ConvNN.add(keras.layers.Convolution1D(256, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(128, (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Convolution1D(64,  (4), strides=(2), activation='relu'))
ConvNN.add(keras.layers.MaxPool1D(2))

ConvNN.add(keras.layers.Flatten())
ConvNN.add(keras.layers.Dense(262144, activation='relu'))
ConvNN.add(keras.layers.Dense(512, activation='relu'))
ConvNN.add(keras.layers.Dense(256,  activation='relu'))

ConvNN.add(keras.layers.Dropout(.5))

ConvNN.add(keras.layers.Dense(2, activation='softmax'))
ConvNN.output_shape

(None, 2)

In [15]:
# callbacksConv = [
#     keras.callbacks.ModelCheckpoint("convNNv2_save_at_{epoch}.h5"),
# ]

In [16]:
callbacksConv = [
    keras.callbacks.ModelCheckpoint("convNNv2_save_at_{epoch}.h5"),
]

In [17]:
# Define the checkpoint path
checkpoint_path = "checkpoint/model2_checkpoint.h5"

import os
# Define the ModelCheckpoint callback
model_checkpoint2 = ModelCheckpoint(
    checkpoint_path,
    save_best_only=False,  # Save only the best model
    monitor='val_loss',  # Monitor validation loss for saving the best model
    mode='min',  # 'min' mode for loss (you can change it to 'max' for accuracy)
    verbose=1  # Print a message when a new best model is saved
)

# Load the model weights if a checkpoint file exists
if os.path.exists(checkpoint_path):
    ConvNN.load_weights(checkpoint_path)

In [18]:
ConvNN.compile( metrics=["accuracy"], 
optimizer = keras.optimizers.Adam(learning_rate=1e-3), 
loss="categorical_crossentropy")

In [19]:
# Continue training from a specific epoch
epochs_completed = 0  # Change this to the number of epochs already completed
total_epochs = 25  # The total number of epochs (including completed ones)
# total_epochs = 25  # The total number of epochs (including completed ones)

ConvNN.fit(
    images, 
    enc_labels, 
    initial_epoch=epochs_completed,
    epochs=total_epochs, 
    callbacks = [model_checkpoint2])

In [20]:
# ConvNN.fit(images, enc_labels, epochs=25, callbacks = callbacksConv)